In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Load Public School Points

In [1]:
school_points = gpd.read_file('../data/processed_data/school_points_with_lcgms.gpkg', dtype={'Zip': str, 'Borough Block Lot': str})

NameError: name 'gpd' is not defined

In [ ]:
public_school_points = school_points[school_points["Managed By Name"]=='DOE'].reset_index(drop=True)

## Load DACs

In [ ]:
# TODO: fix the dtypes in DAC exports so that BBL, ZIP, and other code cols are string
dacs = gpd.read_file('../data/processed_data/dac_nyc_lite.geojson')

### Add Flag to Public Schools if in a DAC

In [ ]:
# check that there aren't any public schools exactly on the border of a DAC
assert public_school_points.geometry.apply(dacs.union_all().covers).sum() == public_school_points.geometry.within(dacs.union_all()).sum()

In [ ]:
public_school_points['in_dac'] = public_school_points.geometry.within(dacs.union_all())

In [ ]:
print("proportion of public schools in a DAC:", public_school_points['in_dac'].sum() / len(public_school_points))

Show Top 10 schools by composite DAC score

In [ ]:
public_schools_with_dacs = gpd.sjoin(public_school_points, dacs, how='left', predicate='within')

In [ ]:
public_schools_with_dacs['dac_designation'] = public_schools_with_dacs['dac_designation'].fillna(False)

In [ ]:
public_schools_with_dacs = public_schools_with_dacs[public_schools_with_dacs['dac_designation']].sort_values('percentile_rank_combined_nyc', ascending=False)

In [ ]:
# TODO: fix this in the processed school points export
public_schools_with_dacs['Zip'] = public_schools_with_dacs['Zip'].apply(lambda x: str(int(x)).zfill(5) if not pd.isna(x) else np.nan)
public_schools_with_dacs['Borough Block Lot'] = public_schools_with_dacs['Borough Block Lot'].apply(lambda x: str(int(x)).zfill(10) if not pd.isna(x) else np.nan)
public_schools_with_dacs['duplicate_geometry'] = public_schools_with_dacs['geometry'].duplicated(keep=False)

In [ ]:
public_school_points[public_school_points['Location Name'].isin(['P.S. 112 Jose Celso Barbosa', 'P.S. 206 Jose Celso Barbosa' ])]

In [ ]:
cols_to_keep_from_schools = ['Location Code', 'Location Name', 'Administrative District Name', 'Principal Name', 'Principal Phone Number', 'Primary Address', 'Zip', 'Borough Block Lot']
cols_to_keep_from_dacs = ['combined_score', 'percentile_rank_combined_nyc', 'burden_score', 'burden_score_percentile', 'vulnerability_score','vulnerability_score_percentile']
all_cols_to_keep = cols_to_keep_from_schools + cols_to_keep_from_dacs
public_schools_with_dacs[all_cols_to_keep + ['geometry', 'duplicate_geometry']].head(25)

In [ ]:
public_schools_with_dacs[all_cols_to_keep + ['geometry']].head(25).explore(tiles='CartoDB positron',
                popup=all_cols_to_keep,
                tooltip=['Location Name'],
                legend=True,
                style_kwds={'fillOpacity': 0.7, 'weight': 1}
)